In [1]:
!pip install --no-cache-dir transformers==4.52.4 torch==2.0.1 datasets==2.13.1 huggingface_hub==0.30.0 peft==0.15.0

ERROR: Could not find a version that satisfies the requirement tokenizer==0.13.3 (from versions: 0.1.1, 0.1.2, 0.1.3, 0.1.10, 0.1.11, 0.1.12, 0.1.14, 1.0.0, 1.0.1, 1.0.2, 1.0.3, 1.0.4, 1.0.5, 1.0.6, 1.0.7, 1.0.8, 1.0.9, 1.1.0, 1.1.1, 1.1.2, 1.2.0, 1.2.1, 1.2.2, 1.2.3, 1.3.0, 1.4.0, 1.4.1, 2.0.0, 2.0.1, 2.0.2, 2.0.3, 2.0.4, 2.0.5, 2.0.6, 2.0.7, 2.1.0, 2.2.0, 2.3.0, 2.3.1, 2.4.0, 2.5.0, 3.0.0, 3.1.0, 3.1.1, 3.1.2, 3.2.0, 3.3.0, 3.3.1, 3.3.2, 3.4.0, 3.4.1, 3.4.2, 3.4.3, 3.4.4, 3.4.5)
ERROR: No matching distribution found for tokenizer==0.13.3


In [29]:
!pip install seqeval accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 52.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 33.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 409.0 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 840.8 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 28.4 MB/s eta 0:00:00
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16162 

In [1]:
import Fine_Tuning
import importlib
importlib.reload(Fine_Tuning)

<module 'Fine_Tuning' from '/content/Fine_Tuning.py'>

In [2]:
#Load CONLL Data
conll_data = Fine_Tuning.load_conll("output.conll")

In [3]:
from datasets import Dataset
dataset = Dataset.from_list(conll_data)
dataset = dataset.train_test_split(test_size=0.1)

In [4]:
from datasets import DatasetDict
dataset = DatasetDict({'train': dataset['train'],'test': dataset['test'],})

In [5]:
#Change to data frame
df = Fine_Tuning.conll_to_dataframe(conll_data)

In [7]:
df.head()

,sentence_id,token,ner_tag
0,0,adidas,B-PRODUCT
1,0,campus,I-PRODUCT
2,0,original,O
3,0,size,O
4,0,36373839,O


In [6]:
dataset

DatasetDict({
    train: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 33
    })
    test: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 4
    })
})

In [7]:
#Label Mapping
unique_tags = set(tag for example in dataset['train']['ner_tags'] for tag in example)
unique_tags = sorted(list(unique_tags))
tag2id = {tag: i for i, tag in enumerate(unique_tags)}
id2tag = {i: tag for tag, i in tag2id.items()}

In [8]:
dataset["train"].features

{'tokens': List(Value('string')), 'ner_tags': List(Value('string'))}

In [9]:
#Tokenization
tokenized_dataset = dataset.map(lambda examples: Fine_Tuning.tokenize_and_align_labels(examples, tag2id, id2tag), batched=True)


Map:   0%|          | 0/33 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Map:   0%|          | 0/4 [00:00<?, ? examples/s]

In [10]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['tokens', 'ner_tags', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 33
    })
    test: Dataset({
        features: ['tokens', 'ner_tags', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 4
    })
})

In [11]:
from transformers import AutoModelForTokenClassification
model = AutoModelForTokenClassification.from_pretrained("xlm-roberta-base", num_labels=len(tag2id),  id2label=id2tag, label2id=tag2id)

Some weights of XLMRobertaForTokenClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
#Training Arguments
from transformers import TrainingArguments
training_args = TrainingArguments(output_dir="./amharic-ner",eval_strategy="epoch",
    learning_rate=2e-5, per_device_train_batch_size=8, per_device_eval_batch_size=8,
    num_train_epochs=3, weight_decay=0.01,logging_dir="./logs",logging_steps=10,
    save_strategy="epoch",report_to='none')

In [13]:
from transformers import AutoTokenizer
model_name = "xlm-roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [16]:
def compute_metrics(p):
    import numpy as np
    from seqeval.metrics import classification_report, f1_score
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)
    true_labels = [[id2tag[l] for l in label if l != -100] for label in labels]
    true_predictions = [[id2tag[p] for (p, l) in zip(prediction, label) if l != -100]
                        for prediction, label in zip(predictions, labels)]
    return {"f1": f1_score(true_labels, true_predictions,zero_division=0),
        "classification_report": classification_report(true_labels, true_predictions,zero_division=0)}


In [ ]:
from transformers import Trainer
trainer = Trainer(model=model,args=training_args, train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],tokenizer=tokenizer,compute_metrics=compute_metrics)
trainer.train()

/tmp/ipython-input-17-4104207816.py:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(model=model,args=training_args, train_dataset=tokenized_dataset["train"],


Epoch,Training Loss,Validation Loss,F1,Classification Report
1,No log,0.456355,0.000000,precision recall f1-score support LOC 0.00 0.00 0.00 4 PRODUCT 0.00 0.00 0.00 4 micro avg 0.00 0.00 0.00 8 macro avg 0.00 0.00 0.00 8 weighted avg 0.00 0.00 0.00 8
2,0.596200,0.353406,0.000000,precision recall f1-score support LOC 0.00 0.00 0.00 4 PRODUCT 0.00 0.00 0.00 4 micro avg 0.00 0.00 0.00 8 macro avg 0.00 0.00 0.00 8 weighted avg 0.00 0.00 0.00 8
3,0.596200,0.335386,0.000000,precision recall f1-score support LOC 0.00 0.00 0.00 4 PRODUCT 0.00 0.00 0.00 4 micro avg 0.00 0.00 0.00 8 macro avg 0.00 0.00 0.00 8 weighted avg 0.00 0.00 0.00 8


TrainOutput(global_step=15, training_loss=0.5468248685201009, metrics={'train_runtime': 725.9008, 'train_samples_per_second': 0.136, 'train_steps_per_second': 0.021, 'total_flos': 6467387072256.0, 'train_loss': 0.5468248685201009, 'epoch': 3.0})